# List out Deployments and Quota
This will create a list of deployments from Azure OpenAI with the usage (Quota). Thanks @ssemyan for the start of this

In [ ]:
import os,requests,json
from azure.identity import DefaultAzureCredential

# Use your credentials to access the subscriptions
api = "https://management.azure.com/.default"
credential = DefaultAzureCredential()
access_token = credential.get_token(api) 
header = {'Authorization': f'Bearer {access_token.token}'}

In [ ]:
#Get a list of subscriptions
subs = requests.get('https://management.azure.com/subscriptions?api-version=2022-12-01',headers=header).json()

# Quota and Deployment
Quota is by Subscription and Region and Model so notice how this grabs that quota per Subscription

> This does tend to timeout or connection error so you might see that too

In [6]:
# Hardcoded to the list of available regions for AOAI
locations=['australiaeast','canadaeast','eastus','eastus2','francecentral','japaneast','northcentralus','southcentralus','swedencentral','switzerlandnorth','uksouth','westeurope']


In [ ]:
items=[]
for sub in subs['value']:
    subId=sub['subscriptionId']

    quota={}
    # Get the quota by the sub and locations
    for location in locations:
        if location in quota:
            continue
        else:
            quota[location]={}
            usages = requests.get(f"https://management.azure.com/subscriptions/{subId}/providers/Microsoft.CognitiveServices/locations/{location}/usages?api-version=2023-05-01",headers=header).json()
            for usage in usages['value']:
                if usage['name']['value'].startswith("OpenAI.Standard."):
                    quota_name = usage['name']['value'].replace("OpenAI.Standard.", "").lower()
                    quota[location][quota_name] = usage['limit']

    uri=f"https://management.azure.com/subscriptions/{subId}/resourceGroups?api-version=2022-12-01"
    rgs = requests.get(f"https://management.azure.com/subscriptions/{subId}/resourceGroups?api-version=2022-12-01",headers=header).json()
    for rg in rgs['value']:
        resourceGroupName=rg['name']
        location=rg['location']
        usage_lookup = {}

        print("rg=",rg['name'],rg['location'])

        # get a list of accounts
        accounts = requests.get(f"https://management.azure.com/subscriptions/{subId}/resourceGroups/{resourceGroupName}/providers/Microsoft.CognitiveServices/accounts?api-version=2023-05-01",headers=header).json()
        for account in accounts['value']:
            accountName=account['name']
            print(f"Account: {accountName}")
            
            deployments = requests.get(f"https://management.azure.com/subscriptions/{subId}/resourceGroups/{resourceGroupName}/providers/Microsoft.CognitiveServices/accounts/{accountName}/deployments?api-version=2023-05-01",headers=header).json()
            for deployment in deployments['value']:

                name=deployment['name']
                model=deployment['properties']['model']['name']
                for rl in deployment['properties']['rateLimits']:
                    if rl['key'] == 'token':
                        tpm = rl['count']

                qt=None
                if location in quota:
                    if model in quota[location]:
                        qt = quota[location][model]*1000
                items.append({'subscriptionId':subId,'resourceGroupName':resourceGroupName,'accountName':accountName,'deploymentName':name,'model':model,'tpm':tpm,'quota':qt,'location':location})


In [ ]:
# Output this to a tab delimited file
with open('models.csv', 'w') as file:
    for item in items:
        file.write(f"{item['subscriptionId']}\t{item['resourceGroupName']}\t{item['accountName']}\t{item['deploymentName']}\t{item['model']}\t{item['tpm']}\t{item['quota']}\t{item['location']}\n")
